# Used Car Price Prediction using kNN and DNN 

In [ ]:
import pandas as pd
print('success')

## Dataset Pre-Processing

### 100,000 UK Used Car Data set

#### Importing the data
audi = pd.read_csv('datasets\UK_Used_Car_Set\audi.csv')
bmw = pd.read_csv('datasets\UK_Used_Car_Set\bmw.csv')
cclass = pd.read_csv('datasets\UK_Used_Car_Set\cclass.csv')
focus = pd.read_csv('datasets\UK_Used_Car_Set\focus.csv')
ford = pd.read_csv('datasets\UK_Used_Car_Set\ford.csv')
hyundi = pd.read_csv('datasets\UK_Used_Car_Set\hyundi.csv')
merc = pd.read_csv('datasets\UK_Used_Car_Set\merc.csv')
skoda = pd.read_csv('datasets\UK_Used_Car_Set\skoda.csv')
toyota = pd.read_csv('datasets\UK_Used_Car_Set\toyota.csv')
vauxhall = pd.read_csv('datasets\UK_Used_Car_Set\vauxhall.csv')
vw = pd.read_csv('datasets\UK_Used_Car_Set\vw.csv')

Known things to remove in pre-processing
* Engine size 0 - supposed to be for electric cars, if enough values are found this could be removed and made into it's own dataset